##  자동 기상 스크래핑

#### ※ 웹 크롤링 및 스크래핑, csv 저장이 동시에 이뤄지기 때문에 작업 중지를 방지하기 위해 단독 실행을 추천합니다.

### Chromedriver

In [ ]:
"""
크롬 드라이버는 사용자의 크롬 버전에 맞게 드라이버를 다운로드 받아 스크래핑 코드와 같은 폴더에 넣어주시면 됩니다. 

크롬 브라우저의 버전은 우측 상단 더보기란 -> 도움말 -> Chrome 정보 를 선택하면 확인할 수 있습니다.

https://chromedriver.chromium.org/downloads 에 가시면 버전에 맞는 크롬 드라이버를 찾아서 다운로드할 수 있습니다.
"""

### Install Selenium

In [13]:
!pip3 install selenium

  Using cached trio_websocket-0.9.2-py3-none-any.whl (16 kB)
  Using cached outcome-1.1.0-py2.py3-none-any.whl (9.7 kB)


### 화면 가로 확장

In [1]:
# 화면 가로 확장 코드 (기본 width 50%)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
"""
변수 h => 하루를 30분 단위로 쪼갠 온도 개수, 데이터의 개수를 h로 설정해서 반복시켰는데, 
            48까지 못 채우는 경우도 있음.

변수 x => 하루 데이터의 시작 시각인 11:50 PM보다 먼저 이른 시각부터 시작했을 경우 중에 11:50 PM 의 데이터가
            결손되었을 경우 반복을 통해 비결손 데이터전까지 처리를 하기 위한 변수.
        
변수 h3 => 2.1.1.1.의 경우에서 하루 동안 48개의 데이터를 스크래핑하기 위한 카운트 변수.

변수 z => 2.1.1.2.의 경우에서 비결손 정상 데이터 지점까지 결손된 시각의 반복 처리 횟수를 결정하기 위한 변수.

======================================================================================================================================================

1. 반복을 위한 month 딕셔너리와 온도를 저장하는 글로벌 변수 지정
    -> 각 달마다 일수가 다르고 하루 단위로 동작되는 코드는 중복되는 내용이 많기 때문에 간소화를 위해 월 Dict 이용.
        
2. 해당 월의 일수 동안 반복하도록 하는 월단위 while문 진입
    -> 연, 월, 일 변수의 조합으로 해당 날짜(최소 단위 1일)의 기상 주소 접속.
    -> 변수 h, h2 초기화.
    
    2.1. 11:50 PM 이전 시각부터 시작하는 경우
            -> 42는 적당히 오차 범위일거라 생각하는 숫자로 임의 지정한 것이지만 42 ~ 48 까지는 정상 시각 시작전을 뜻함.
            -> 변수 x 초기화.
        
        2.1.1. 하루의 기상 데이터 시작 시각의 경우에 따른 while문 진입
                -> 변수 x의 값에 따라 경우가 나뉘는데 크게 x==1인 경우(2.1.1.1.)와 x>=2인 경우(2.1.1.2.)로 나눠서 설명.
                -> 중복이 많아서 clk_11_48[clk_str]로 처리하려 했으나 이전 시각대에서 47같은 숫자가 나와서 하나씩 따로 설정함.   
                
            2.1.1.1. 11:50 PM의 이전 시각부터 해당일의 데이터가 시작하고, 11:50 PM의 데이터도 존재할 경우
                       -> h3 변수 지정.
                       -> 11:50 PM을 기준으로 하루 48개의 데이터를 스크래핑하는 while문 진입.
                       
            ---------------------------------------------------------------------------------------------------------------------------------
            |    2.1.1.1.1. 사이트에서 시각과 온도를 스크랩하도록 Try문 실행
            |                -> 5:01 과 같은 규칙 외 시각을 걸러내기 위해 dict 안에 있는지 검사.
            |                   이상 시각 검출시에는 변수 h2를 1만 증가시켜서 다음 시각으로 이동.
            |            
            |        2.1.1.1.1.1. 순차적으로 증가중인 h3와 현재 시각 문자열을 clk순번 dict 출력 숫자로 비교해봤을때 일치하는 경우 (=시각이 정상적인 경우)
 <Core>     |                    -> 시각, 순번, 온도 정보로 csv 파일 저장 후 변수 h2, h3 증가.
            |                    
            |        2.1.1.1.1.2. 다음 순서로 올 h3와 현재 시각이 일치하지 않는 경우 (=데이터 결손)
            |                    -> 결손 데이터 칸의 정상시각을 입력후 전 시각의 온도를 그대로 저장.
            |                    
            |    2.1.1.1.2. 정상 데이터수를 채우지 못하고 NoSuchElementException 로 인해 조기 종료될 경우
            |                -> 예외 처리를 통하여 해당 경우로 진입하고 변수 h가 48이 될때까지(=하루 데이터 갯수 충족) 결손된 데이터를 정상시각까지 처리.
            ---------------------------------------------------------------------------------------------------------------------------------              
            
            2.1.1.2. 11:50 PM의 이전 시각부터 해당일의 데이터가 시작하고, 11:50 PM의 데이터가 존재하지 않을 경우
                        -> 결손데이터가 지속되다가 데이터가 존재하는 지점까지 가게 되는데 이때 변수 z를 이용해 정상지점까지 처리 반복.
                        -> h3 = 1 + (x-1)의 계산으로 결손 데이터를 반복한뒤 직후 지점부터 하루 끝 지점까지 반복하도록 설정.
                
                2.1.1.2.1. 이후 동작은 <Core>와 동일한 구조로 실행
             
    2.2. 11:50 PM 부터 하루 데이터가 시작하는 경우
        <Core> 동작 실행. 시작 시각이 정상적이기 때문에 중간 결손데이터 처리만 하면 되므로 단계가 간단함.
    
"""

'\n변수 h => 하루를 30분 단위로 쪼갠 온도 개수, 데이터의 개수를 h로 설정해서 반복시켰는데, \n            48까지 못 채우는 경우도 있음.\n\n변수 x => 하루 데이터의 시작 시각인 11:50 PM보다 먼저 이른 시각부터 시작했을 경우 중에 11:50 PM 의 데이터가\n            결손되었을 경우 반복을 통해 비결손 데이터전까지 처리를 하기 위한 변수.\n        \n변수 h3 => 2.1.1.1.의 경우에서 하루 동안 48개의 데이터를 스크래핑하기 위한 카운트 변수.\n\n변수 z => 2.1.1.2.의 경우에서 비결손 정상 데이터 지점까지 결손된 시각의 반복 처리 횟수를 결정하기 위한 변수.\n\n======================================================================================================================================================\n\n1. 반복을 위한 month 딕셔너리와 온도를 저장하는 글로벌 변수 지정\n    -> 각 달마다 일수가 다르고 하루 단위로 동작되는 코드는 중복되는 내용이 많기 때문에 간소화를 위해 월 Dict 이용.\n        \n2. 해당 월의 일수 동안 반복하도록 하는 월단위 while문 진입\n    -> 연, 월, 일 변수의 조합으로 해당 날짜(최소 단위 1일)의 기상 주소 접속.\n    -> 변수 h, h2 초기화.\n    \n    2.1. 11:50 PM 이전 시각부터 시작하는 경우\n            -> 42는 적당히 오차 범위일거라 생각하는 숫자로 임의 지정한 것이지만 42 ~ 48 까지는 정상 시각 시작전을 뜻함.\n            -> 변수 x 초기화.\n        \n        2.1.1. 하루의 기상 데이터 시작 시각의 경우에 따른 while문 진입\n                -> 변수 x의 값에 따라 경우가 

### 실행 코드

In [ ]:
import time
import os
import csv

print('※ 웹 크롤링 및 스크래핑, csv 저장이 동시에 이뤄지기 때문에 작업 중지를 방지하기 위해 단독 실행을 추천합니다.\n')
time.sleep(2)

print('※ 크롬 드라이버는 사용자의 크롬 버전에 맞게 드라이버를 다운로드 받아 스크래핑 코드와 같은 폴더에 넣어주시면 됩니다.\n')
time.sleep(2)

print('※ 크롬 브라우저의 버전은 우측 상단 더보기란 -> 도움말 -> Chrome 정보 를 선택하면 확인할 수 있습니다.\n')
time.sleep(2)

print('※ https://chromedriver.chromium.org/downloads 에 가시면 버전에 맞는 크롬 드라이버를 찾아서 다운로드할 수 있습니다.\n')
time.sleep(2)

print('※ 저장을 위한 폴더 생성 \n')
try:
    os.mkdir('./Weather_Data_CSV')
except FileExistsError:
    print('<!> 해당 폴더가 이미 존재합니다.\n폴더를 확인해주세요.\n')
    
print('※ Install Selenium \n')
time.sleep(2)
!pip3 install selenium

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException


clk_11_48 = {'11:50 PM':1, '12:20 AM':2, '12:50 AM':3, '1:20 AM':4, '1:50 AM':5, '2:20 AM':6,
         '2:50 AM':7, '3:20 AM':8, '3:50 AM':9, '4:20 AM':10, '4:50 AM':11, '5:20 AM':12,
         '5:50 AM':13, '6:20 AM':14, '6:50 AM':15, '7:20 AM':16, '7:50 AM':17, '8:20 AM':18,
         '8:50 AM':19, '9:20 AM':20, '9:50 AM':21, '10:20 AM':22, '10:50 AM':23, '11:20 AM':24,
         '11:50 AM':25, '12:20 PM':26, '12:50 PM':27, '1:20 PM':28, '1:50 PM':29, '2:20 PM':30,
         '2:50 PM':31, '3:20 PM':32, '3:50 PM':33, '4:20 PM':34, '4:50 PM':35, '5:20 PM':36,
         '5:50 PM':37, '6:20 PM':38, '6:50 PM':39, '7:20 PM':40, '7:50 PM':41, '8:20 PM':42,
         '8:50 PM':43, '9:20 PM':44, '9:50 PM':45, '10:20 PM':46, '10:50 PM':47, '11:20 PM':48}

re_clk_11_48 = dict(map(reversed, clk_11_48.items()))

def Monthly_Temp(YYYY, MM):
    driver.implicitly_wait(10)
    YYYY = y
    MM = m

    # 1.
    month = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}
    d=0
    
    global temp_t_x
    global temp_t
    
    global humi_h_x
    global humi_h
    
    # 2.
    while(d<month[m]):
        d=d+1
        driver.get('https://www.wunderground.com/history/daily/gb/london/EGLC/date/'+str(y)+'-'+str(m)+'-'+str(d))
        driver.implicitly_wait(10)
        time.sleep(3)
                
        h = 1 
        h2 = 1
        
        clk = driver.find_element(By.XPATH,'//*[@id="inner-content"]/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table/tbody/tr['+str(h2)+']/td[1]/span')
        clk_str = clk.text
        clk_str.split()

        # 2.1. 
        if(int(clk_11_48[clk_str])>=42):
            x = 0
            
            # 2.1.1.
            while(h2 < 49):
                clk = driver.find_element(By.XPATH,'//*[@id="inner-content"]/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table/tbody/tr['+str(h2)+']/td[1]/span')
                clk_str = clk.text
                clk_str.split()

                if(clk_str == '11:50 PM'):
                    x = 1
                if(clk_str == '12:20 AM'):
                    x = 2
                if(clk_str == '12:50 AM'):
                    x = 3
                if(clk_str == '1:20 AM'):
                    x = 4  
                if(clk_str == '1:50 AM'):
                    x = 5
                if(clk_str == '2:20 AM'):
                    x = 6
                if(clk_str == '2:50 AM'):
                    x = 7
                if(clk_str == '3:20 AM'):
                    x = 8
                if(clk_str == '3:50 AM'):
                    x = 9
                if(clk_str == '4:20 AM'):
                    x = 10

                # 2.1.1.1.
                if(x == 1):
                    h3 = 1
                    
                    while (h3 < 49):
                        
                        # 2.1.1.1.1.
                        try:
                            clk2 = driver.find_element(By.XPATH,'//*[@id="inner-content"]/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table/tbody/tr['+str(h2)+']/td[1]/span')
                            clk_str2 = clk2.text
                            clk_str2.split()
                            
                            if(clk_str2 in clk_11_48):
                                
                                # 2.1.1.1.1.1.
                                if(h3==clk_11_48[clk_str2]):
                                    date_list = [int(y), int(m), int(d)] 
                                    date_list.append(clk_str2) 
                                    date_list.append(int(h3)) 
                                    
                                    temp = driver.find_element(By.XPATH,'//*[@id="inner-content"]/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table/tbody/tr['+str(h2)+']/td[2]')
                                    temp_str = temp.text 
                                    temp_str.split
                                    temp_t=temp_str[:2] 
                                    temp_t_x = temp_t
                                    
                                    humi = driver.find_element(By.XPATH,'//*[@id="inner-content"]/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table/tbody/tr['+str(h2)+']/td[4]')
                                    humi_str = humi.text
                                    humi_h=humi_str[:3]
                                    humi_h_x = humi_h
                                    
                                    weather_list = date_list 
                                    weather_list.append(temp_t_x)
                                    weather_list.append('℃') 
                                    weather_list.append(humi_h_x) 
                                    weather_list.append('%')
                                    
                                    time.sleep(0.02)
                                    
                                    with open("./Weather_Data_CSV/"+str(y)+'_'+str(m)+'_'+"London_Temperature.csv", 'a', newline='') as file:
                                        writer = csv.writer(file)
                                        writer.writerow(weather_list)
                                    h2 = h2+1
                                    h3 = h3+1
                                    
                                # 2.1.1.1.1.2.
                                else:
                                    date_list = [int(y), int(m), int(d)]
                                    date_list.append(re_clk_11_48[h])
                                    date_list.append(int(h3))
                                    na_list = date_list
                                    na_list.append(temp_t) 
                                    na_list.append('℃')
                                    na_list.append(humi_h) 
                                    na_list.append('%')
                                    h3 = h3+1

                                    with open("./Weather_Data_CSV/"+str(y)+'_'+str(m)+'_'+"London_Temperature.csv", 'a', newline='') as file:
                                        writer = csv.writer(file)
                                        writer.writerow(na_list)

                                h = h+1 
                                
                            else:
                                for E in range(1):
                                    h2 = h2+1
                                    
                        # 2.1.1.1.2.
                        except NoSuchElementException:
                            if(h<49):
                                date_list = [int(y), int(m), int(d)]
                                date_list.append(re_clk_11_48[h])
                                date_list.append(int(h))
                                na_list = date_list
                                na_list.append(temp_t) 
                                na_list.append('℃')
                                na_list.append(humi_h) 
                                na_list.append('%')

                                with open("./Weather_Data_CSV/"+str(y)+'_'+str(m)+'_'+"London_Temperature.csv", 'a', newline='') as file:
                                    writer = csv.writer(file)
                                    writer.writerow(na_list)
                                h = h+1
                            else:
                                break
                    break
                
                # 2.1.1.2.
                if(x >= 2):
                    z = 1
                    while(z < x):
                        date_list = [int(y), int(m), int(d)]
                        date_list.append(re_clk_11_48[h])
                        date_list.append(z)
                        na_list = date_list
                        na_list.append(temp_t_x) 
                        na_list.append('℃')
                        na_list.append(humi_h_x) 
                        na_list.append('%')

                        with open("./Weather_Data_CSV/"+str(y)+'_'+str(m)+'_'+"London_Temperature.csv", 'a', newline='') as file:
                            writer = csv.writer(file)
                            writer.writerow(na_list)
                        z = z+1
                        h = h+1
                        
                    h3 = 1 + (x-1)
                    
                    # 2.1.1.2.1.
                    while (h3 < 49):
                        try:
                            clk2 = driver.find_element(By.XPATH,'//*[@id="inner-content"]/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table/tbody/tr['+str(h2)+']/td[1]/span')
                            clk_str2 = clk2.text
                            clk_str2.split()
                            
                            if(clk_str2 in clk_11_48):
                                if(h3==clk_11_48[clk_str2]):
                                    date_list = [int(y), int(m), int(d)]
                                    date_list.append(clk_str2) 
                                    date_list.append(int(h3)) 
                                                                        
                                    temp = driver.find_element(By.XPATH,'//*[@id="inner-content"]/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table/tbody/tr['+str(h2)+']/td[2]')
                                    temp_str = temp.text 
                                    temp_str.split
                                    temp_t=temp_str[:2] 
                                    temp_t_x = temp_t
                                    
                                    humi = driver.find_element(By.XPATH,'//*[@id="inner-content"]/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table/tbody/tr['+str(h2)+']/td[4]')
                                    humi_str = humi.text
                                    humi_h=humi_str[:3]
                                    humi_h_x = humi_h
                                    
                                    weather_list = date_list 
                                    weather_list.append(temp_t_x) 
                                    weather_list.append('℃')
                                    weather_list.append(humi_h_x) 
                                    weather_list.append('%')
                                    
                                    time.sleep(0.02)

                                    with open("./Weather_Data_CSV/"+str(y)+'_'+str(m)+'_'+"London_Temperature.csv", 'a', newline='') as file:
                                        writer = csv.writer(file)
                                        writer.writerow(weather_list)

                                    h2 = h2+1
                                    h3 = h3+1

                                else:
                                    try:
                                        date_list = [int(y), int(m), int(d)]
                                        date_list.append(re_clk_11_48[h])
                                        date_list.append(int(h3))
                                        na_list = date_list
                                        na_list.append(temp_t) 
                                        na_list.append('℃')
                                        na_list.append(humi_h) 
                                        na_list.append('%')
                                        h3 = h3+1

                                        with open("./Weather_Data_CSV/"+str(y)+'_'+str(m)+'_'+"London_Temperature.csv", 'a', newline='') as file:
                                            writer = csv.writer(file)
                                            writer.writerow(na_list)

                                    except KeyError:
                                        break

                                h = h+1
                            
                            else:
                                for E in range(1):
                                    h2 = h2+1

                        except NoSuchElementException:
                            if(h<49):
                                date_list = [int(y), int(m), int(d)]
                                date_list.append(re_clk_11_48[h])
                                date_list.append(int(h))
                                na_list = date_list
                                na_list.append(temp_t)
                                na_list.append('℃')
                                na_list.append(humi_h)
                                na_list.append('%')

                                with open("./Weather_Data_CSV/"+str(y)+'_'+str(m)+'_'+"London_Temperature.csv", 'a', newline='') as file:
                                    writer = csv.writer(file)
                                    writer.writerow(na_list)
                                h = h+1
                            else:
                                break
                    break
                    
                else:
                    h2=h2+1
                    
                    
        # 2.2.
        else:
            while (h2 < 49):
                try:    
                    clk = driver.find_element(By.XPATH,'//*[@id="inner-content"]/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table/tbody/tr['+str(h2)+']/td[1]/span')
                    clk_str = clk.text
                    clk_str.split()

                    if(clk_str in clk_11_48):
                        if(h==clk_11_48[clk_str]):
                            date_list = [int(y), int(m), int(d)] 
                            date_list.append(clk_str) 
                            date_list.append(int(h)) 
                            temp = driver.find_element(By.XPATH,'//*[@id="inner-content"]/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table/tbody/tr['+str(h2)+']/td[2]')
                            temp_str = temp.text 
                            temp_str.split
                            temp_t=temp_str[:2]
                            temp_t_x = temp_t

                            humi = driver.find_element(By.XPATH,'//*[@id="inner-content"]/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table/tbody/tr['+str(h2)+']/td[4]')
                            humi_str = humi.text
                            humi_h=humi_str[:3]
                            humi_h_x = humi_h

                            weather_list = date_list 
                            weather_list.append(temp_t_x) 
                            weather_list.append('℃') 
                            weather_list.append(humi_h_x) 
                            weather_list.append('%')
                            time.sleep(0.02)

                            with open("./Weather_Data_CSV/"+str(y)+'_'+str(m)+'_'+"London_Temperature.csv", 'a', newline='') as file:
                                writer = csv.writer(file)
                                writer.writerow(weather_list)

                            h2 = h2+1
                        
                        else:
                            try:
                                date_list = [int(y), int(m), int(d)]
                                date_list.append(re_clk_11_48[h])
                                date_list.append(int(h))
                                na_list = date_list
                                na_list.append(temp_t)
                                na_list.append('℃')
                                na_list.append(humi_h)
                                na_list.append('%')

                                with open("./Weather_Data_CSV/"+str(y)+'_'+str(m)+'_'+"London_Temperature.csv", 'a', newline='') as file:
                                    writer = csv.writer(file)
                                    writer.writerow(na_list)
                                    
                            except KeyError:
                                break
                        h = h+1
                    
                    else:
                        for E in range(1):
                            h2 = h2+1

                except NoSuchElementException:
                    if(h<49):
                        date_list = [int(y), int(m), int(d)]
                        date_list.append(re_clk_11_48[h])
                        date_list.append(int(h))
                        na_list = date_list
                        na_list.append(temp_t)
                        na_list.append('℃')
                        na_list.append(humi_h)
                        na_list.append('%')
                            
                        with open("./Weather_Data_CSV/"+str(y)+'_'+str(m)+'_'+"London_Temperature.csv", 'a', newline='') as file:
                            writer = csv.writer(file)
                            writer.writerow(na_list)
                        h = h+1
                        
                    else:
                        break
                        
        temp_t_x = temp_t                
        humi_h_x = humi_h
        
path = "./chromedriver.exe"
driver = webdriver.Chrome(path)
driver.implicitly_wait(10)

driver.get('https://www.wunderground.com/history/daily/gb/london/EGLC/date/2021-1-1')
time.sleep(1)

driver.find_element(By.XPATH, '//*[@id="wuSettings"]/i').click()
time.sleep(1)

driver.find_element(By.XPATH, '//*[@id="wuSettings-quick"]/div/a[2]').click()
time.sleep(1)

for y in range(2013, 2017):
    
    for m in range(1, 13):   
        Monthly_Temp(y, m)
        
        temp_t_x = temp_t
        humi_h_x = humi_h
            

print('※ 스크래핑 종료.')
driver.quit()

※ 웹 크롤링 및 스크래핑, csv 저장이 동시에 이뤄지기 때문에 작업 중지를 방지하기 위해 단독 실행을 추천합니다.

※ 크롬 드라이버는 사용자의 크롬 버전에 맞게 드라이버를 다운로드 받아 스크래핑 코드와 같은 폴더에 넣어주시면 됩니다.

※ 크롬 브라우저의 버전은 우측 상단 더보기란 -> 도움말 -> Chrome 정보 를 선택하면 확인할 수 있습니다.

※ https://chromedriver.chromium.org/downloads 에 가시면 버전에 맞는 크롬 드라이버를 찾아서 다운로드할 수 있습니다.

※ 저장을 위한 폴더 생성 

<!> 해당 폴더가 이미 존재합니다.
폴더를 확인해주세요.

※ Install Selenium 



You should consider upgrading via the 'C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.
C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_12816/3639861271.py:382: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)
